In [1]:
import torch
import torchvision
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from time import time

# set seed for the random number generations (to be able to replicate the results)
random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [2]:
# loading and transforming the data

# transform
transform = transforms.Compose([transforms.ToTensor(), #transforms image into colour matrices with values ranging from 0 -1 (becomes a Torch Tensor)
                              transforms.Normalize((0.5,), (0.5,)), # normalizes tensor
                              ])

# loading the data
trainset = datasets.EMNIST('\Datasets\MNIST\Trainset', download=True, split='letters', train=True, transform=transform)
valset = datasets.EMNIST('\Datasets\MNIST\Valset', download=True, split='letters', train=False, transform=transform)

# creating the batches 
train_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True) 
test_loader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=True) 

In [3]:
# defining the neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,10,5) # for convolution: imput size has to match number of channels for the image
        self.conv2 = nn.Conv2d(10,20,5)
        self.lin1 = nn.Linear(320, 27) # 5760 is calculated by: print(x.view(64, -1).shape)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(F.dropout(self.conv2(x), training=self.training), 2)) # defined in forward part, because hear during training can be specified (not done with test)
        x = x.view(64, -1) # transforms to 2D. Returns a new tensor with the same data as the self tensor but of a different shape.Keep 64 batches, calculate / infer other dimension
        x = F.relu(self.lin1(x)) # requires 2D input
        return F.log_softmax(x)
        
model = Net()
print(model)

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (lin1): Linear(in_features=320, out_features=27, bias=True)
)


In [4]:
# # Print shape of images at different points in the model
# print(images.shape)
# model = nn.Conv2d(1,10,5)
# model1b = nn.Dropout2d()
# output_pooled = F.max_pool2d(model1b(model(images)),2)
# print(output_pooled.shape)

# model2 = nn.Conv2d(10,20,5)

# output2_pooled = F.max_pool2d(model1b(model2(output_pooled)),2)
# print(output2_pooled.shape)

# print(output2_pooled.view(64, -1).shape)

In [5]:
# defining the optimization method
learning_rate = 0.003
momentum = 0.9
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                      momentum=momentum)

In [6]:
# setting the image to the correct size
images, labels = next(iter(train_loader))
#images = images.view(images.shape[0], -1)
print(images.shape)
# images = images.transpose(1,3)

# criterion = nn.NLLLoss()
# logps = model(images) #log probabilities
# loss = criterion(logps, labels) #calculate the NLL loss

torch.Size([64, 1, 28, 28])


In [7]:
# training the neural network
# core training process (NN iterates over the training set and updates the weights))
# time0 = time()
epochs = 5
def train(epochs):
    #for e in range(epochs):
#     time0 = time()
        running_loss = 0
        correct = 0
        for images, labels in train_loader:
            # Training pass
            optimizer.zero_grad() # sets all optimized gradients to zero

            output = model(images)
            
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(labels.data.view_as(pred)).sum()
            
            #loss 
            loss = F.nll_loss(output, labels)

            #This is where the model learns by backpropagating
            loss.backward()

            #And optimizes its weights here
            optimizer.step()

            running_loss += loss.item()
        else: 
            print("Epoch {} - Training loss: {:.4f} Train Accuracy: ({:.2f}%)".format(
                i+1, running_loss/len(train_loader),
                100. * correct / len(train_loader.dataset)))
#     print("\nTraining Time (in minutes) =",(time()-time0)/60)

In [8]:
test_losses = []
def test():
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for images, labels in test_loader:
      output = model(images)
      test_loss += F.nll_loss(output, labels, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(labels.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print("Test los: {:.4f}, Accuracy: {}/{} ({:.2f}%)".format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))
# test()

In [9]:
time0 = time()
test()
for i in range(epochs):
  train(i)
  test()
print("\nTraining Time (in minutes) =",(time()-time0)/60)

<ipython-input-3-7882ed6ee58f>:14: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
C:\Users\Lina_\anaconda3\lib\site-packages\torch\nn\_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Test los: 3.3102, Accuracy: 186/20800 (0.89%)
Epoch 1 - Training loss: 1.2092 Train Accuracy: (67.47%)
Test los: 0.6590, Accuracy: 16791/20800 (80.73%)
Epoch 2 - Training loss: 0.6144 Train Accuracy: (82.06%)
Test los: 0.5747, Accuracy: 17325/20800 (83.29%)
Epoch 3 - Training loss: 0.5479 Train Accuracy: (83.71%)
Test los: 0.4331, Accuracy: 18096/20800 (87.00%)
Epoch 4 - Training loss: 0.3919 Train Accuracy: (88.03%)
Test los: 0.2881, Accuracy: 18902/20800 (90.88%)
Epoch 5 - Training loss: 0.2562 Train Accuracy: (91.67%)
Test los: 0.2638, Accuracy: 19087/20800 (91.76%)

Training Time (in minutes) = 12.990006347497305


In [10]:
# save the model
torch.save(model, './04b_model.pt') 
#  save the trained model